## IMPORT LIBRARIES

In [1]:
import numpy as np 
import pandas as pd

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from TurkishStemmer import TurkishStemmer

from sklearn.metrics import classification_report

In [2]:
pd.set_option('display.max_colwidth', 120)

## LOAD DATASET

In [3]:
df = pd.read_excel("reviews.xlsx")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   YORUM   175 non-null    object
 1   DUYGU   175 non-null    object
dtypes: object(2)
memory usage: 2.9+ KB


In [5]:
df.head()

,YORUM,DUYGU
0,"Satıcı ve buna aracı oldukları için Amazon bilsin ki, satılan ürün sahte/replika her ne diyorsanız, ORJİNAL DEĞİL. U...",olumsuz
1,Xiaomi Kulaklık Redmi Earbuds SÜründe ara ara ses kesiliyor bide kulaklığın tıkaç ını çıkardım seri no yok sahte ür...,olumsuz
2,Ürün başlığına ürünün orjinal olmadığını yazarsanız insanlar buna göre alışveriş yapar tabii kasti bir şekilde yapıl...,olumsuz
3,ürün elime ulaştığında kullanılmış bir haldeydi neyse sıkıntı yapmadım orjinalliğini sorgulamadım bi de ne göreyim ü...,olumsuz
4,"Ürün replika ürün arkadaşlar orjinal değil bilginiz olsun,kulaklıklar da seri numarası yok,kutudaki seri numarasıda ...",olumsuz


In [6]:
df[75:80]

,YORUM,DUYGU
75,orta kalite ve evin dışındaki ses seviyesi düşük,cekimser
76,Fiyatına göre iyi bir ürün ancak çok fazla beklentiniz olmasın.,cekimser
77,Fiyatına göre harika iş çıkarıyor. Sağlam bir müzik kulağınız varsa tatmin etmeyebilir. Hifi performans beklemeyin.,cekimser
78,Ürün malum.Ancak kutu içeriğinde yazmasına rağmen şarj kablosu yok.,cekimser
79,Ürün ithalatçı firma garantili ve bazı yönlerine göre orijinal gözüküyor bazı yönlerine göre sahte gözüküyor bunu be...,cekimser


In [7]:
df.tail()

,YORUM,DUYGU
170,Gayet güzel bir kulaklık fırsatınız varsa eğer çok çok alıp yedekleyin.,olumlu
171,Fiyat-Performans ürünü olarak oldukça iyi.,olumlu
172,ürünü aldım çok kısa sürede ulaştı. ürü iyi bir ürün iPhone telefonumda kullanıyorum. gayet güzel bir ürün,olumlu
173,Öncekinin kutusunu kaybettiğim için yenisini aldım. Bir daha kaybetsem yine bunu alırım. Tam bir fiyat performans ca...,olumlu
174,1 Haftadır kullanıyorum ve çok memnunum. Şarjı da söylendiği gibi çabuk bitmiyor. Tavsiye ederim.,olumlu


In [8]:
df['DUYGU'].value_counts()

olumlu      80
olumsuz     70
cekimser    25
Name: DUYGU, dtype: int64

In [9]:
X = list(df['YORUM'])
y = list(df['DUYGU'].map({'olumsuz': 0, 'cekimser': 1, 'olumlu': 2}))

In [10]:
# Instead of using sklearn train-test split, I've chosen the test data manually 
# (to-distribute evenly for each class)
X_train = X[0:60]  + X[70:90] + X[95:160]
X_test  = X[60:70] + X[90:95] + X[160:175]
y_train = y[0:60]  + y[70:90] + y[95:160]
y_test  = y[60:70] + y[90:95] + y[160:175]

## HELPER FUNCTIONS

In [11]:
def preprocess_review(review):
    '''
    Takes a single review as an input
    Returns a processed, clean (tokenized, puntuations/stop words removed and stemmed) sentence
    '''
    tokenized = word_tokenize(review.lower(), language='turkish')
    no_punc = [t for t in tokenized if t.isalpha()]
    no_stop = [t for t in no_punc if t not in stopwords.words('turkish')]
    
    stemmer = TurkishStemmer()
    review_cleaned = [stemmer.stem(t) for t in no_stop]
    
    return review_cleaned

In [12]:
def frequencies(reviews, labels):
    '''
    Takes list of reviews and sentiment classes as input
    Returns a dictionary that maps each pair of (word, label) to its frequency
    '''
    freqs = {}
    
    for y, review in zip(labels, reviews):
        for word in preprocess_review(review):
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1

    return freqs

In [13]:
def log_probs(freqs, labels):
    '''
    Takes frequencies and sentiment class labels as input
    Returns log-likelihood of each word for each class
    '''
    log_likelihoods = {}
    
    # Number of labels for each class
    y_neg = labels.count(0)
    y_notr = labels.count(1)
    y_pos = labels.count(2)
    
    # Prior probabilities for each class
    logprior_neg = np.log(y_neg / len(labels))
    logprior_notr = np.log(y_notr / len(labels))
    logprior_pos = np.log(y_pos / len(labels))
    log_priors = (logprior_neg, logprior_notr, logprior_pos)
    
    #unique words (vocabulary)
    vocab = set([pair[0] for pair in freqs.keys()]) 
    
    # Total Number of word-frequencies for each class
    N_neg = N_notr = N_pos = 0
    for pair in freqs.keys():
        if pair[1] == 0:
            N_neg += freqs[pair]
        elif pair[1] == 1:
            N_notr += freqs[pair]
        else:
            N_pos += freqs[pair]

    for word in vocab:
        # neg/notr/pos frequency of the word
        freq_neg = freqs.get((word, 0), 0)
        freq_notr = freqs.get((word, 1), 0)
        freq_pos = freqs.get((word, 2), 0)

        # neg/notr/pos probability of the word
        p_neg = (freq_neg + 1) / (N_neg + len(vocab))
        p_notr = (freq_notr + 1) / (N_notr + len(vocab))
        p_pos = (freq_pos + 1) / (N_pos + len(vocab))
        
        # log probabilities (likelihood) of the word for each class
        log_likelihoods[(word, "neg")] = np.log(p_neg)
        log_likelihoods[(word, "notr")] = np.log(p_notr)
        log_likelihoods[(word, "pos")] = np.log(p_pos)
        
    return log_priors, log_likelihoods

In [14]:
def predict_reviews(reviews, y_true, logpriors, log_likelihoods):
    """
    Takes list of reviews, true labels, log-probabilities as input
    Prints the evaluation metric results
    """
    y_preds = []
    
    for review in reviews:
        words = preprocess_review(review)
        
        p_neg = logpriors[0]
        p_notr = logpriors[1]
        p_pos = logpriors[2]
        
        for word in words:    
            p_neg += log_likelihoods.get((word, "neg"), 0)
            p_notr += log_likelihoods.get((word, "notr"), 0)
            p_pos += log_likelihoods.get((word, "pos"), 0)

        y_preds.append(np.argmax(np.array([p_neg, p_notr, p_pos])))

    print(classification_report(np.array(y_true), 
                                np.array(y_preds), 
                                target_names=["olumsuz","cekimser","olumlu"]))
    

## MODEL

In [15]:
# Obtain word frequencies for each class
freqs = frequencies(X_train, y_train)

In [16]:
# Calculate log-likelihoods (train the model)
log_priors, log_likelihoods = log_probs(freqs, y_train)

In [17]:
# Make predictions on test set using the probabilities
predict_reviews(X_test, y_test, log_priors, log_likelihoods)

              precision    recall  f1-score   support

     olumsuz       0.73      0.80      0.76        10
    cekimser       0.00      0.00      0.00         5
      olumlu       0.78      0.93      0.85        15

    accuracy                           0.73        30
   macro avg       0.50      0.58      0.54        30
weighted avg       0.63      0.73      0.68        30



**RESULT**
*  My Model has obtained **73%** accuracy on the test set
*  The result is low because of the limited data (total of 175 reviews (train:145, test:30))